# Preparação da base de dados para treinar os classificadores

In [27]:
from src import (
    Folder,
    GetModel,
    SegmentedList,
    ds2df,
    save_results,
    File2df,
    Best_rf,
    Save_model
    )

from os.path import join
DATA = "Dados"
MODEL = "Models"

In [2]:
#adquindo as imagens e seus respectivos rotulos

dataBasePath = join(DATA,"dados")
images, labels = Folder(dataBasePath)

In [3]:

segmentation_model = GetModel(join("models,best.pt"))
process_images = SegmentedList(images,segmentation_model)

WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 155.6ms
Speed: 4.0ms preprocess, 155.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 (no detections), 139.6ms
Speed: 4.0ms preprocess, 139.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 138.6ms
Speed: 3.0ms preprocess, 138.6ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 112.7ms
Speed: 3.0ms prep

In [4]:
# com as imagens processadas, precisamos descartar as imagens que não foram segmentadas e salvar seu nome em um arquivo com as falhas

fails = []

# Iterando de trás para frente
for i in range(len(process_images) - 1, -1, -1):
    if process_images[i] is None:
        fails.append(labels[i])
        
        # Remove os elementos das listas
        del process_images[i]
        del labels[i]


with open(r"Dados\falhas.txt", "w") as arquivo:
    for fail in fails:
        arquivo.write(fail + "\n")
        
print("Imagens segmentadas com sucesso: ",len(process_images))
print("Rotulos validos: ",len(labels))


Imagens segmentadas com sucesso:  310
Rotulos validos:  310


# Extraindo as Caracteristicas de interesse da base de dados

In [5]:
df = ds2df(process_images,labels)

### Visando evitar repetir o processamento para a mesma base de dados podemos exportar a mesma para um arquivo e apenas ler a partir do arquivo quando quisermos novamente

In [16]:
# vamos registrar os datasets por dada de sua geração
from datetime import datetime
data = str(datetime.today()).split(" ")[0].split("-")
data = data[2] + '-' + data[1] + '-' + data[0]
data

'12-03-2024'

In [ ]:

save_results(df,DATA,data,".csv")

# Treinando os classificadores

In [24]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
df = File2df(join(DATA,"11-13-2024.csv"))

df.drop("Unnamed: 0",axis=1,inplace=True)
df.columns

.csv


Index(['Name', 'FAMACHA', 'Mean_R', 'Mean_G', 'Mean_B', 'Median_R', 'Median_G',
       'Median_B', 'Std_R', 'Std_G', 'Std_B'],
      dtype='object')

In [25]:
model = Best_rf(df,join("docs","classification"))


 Teste usando 10 & 100 Realizado!

 Teste usando 10 & 200 Realizado!

 Teste usando 10 & 300 Realizado!

 Teste usando 10 & 400 Realizado!

 Teste usando 10 & 500 Realizado!

 Teste usando 10 & 600 Realizado!

 Teste usando 10 & 700 Realizado!

 Teste usando 10 & 800 Realizado!

 Teste usando 10 & 900 Realizado!

 Teste usando 10 & 1000 Realizado!

 Teste usando 20 & 100 Realizado!

 Teste usando 20 & 200 Realizado!

 Teste usando 20 & 300 Realizado!

 Teste usando 20 & 400 Realizado!

 Teste usando 20 & 500 Realizado!

 Teste usando 20 & 600 Realizado!

 Teste usando 20 & 700 Realizado!

 Teste usando 20 & 800 Realizado!

 Teste usando 20 & 900 Realizado!

 Teste usando 20 & 1000 Realizado!

 Teste usando 30 & 100 Realizado!

 Teste usando 30 & 200 Realizado!

 Teste usando 30 & 300 Realizado!

 Teste usando 30 & 400 Realizado!

 Teste usando 30 & 500 Realizado!

 Teste usando 30 & 600 Realizado!

 Teste usando 30 & 700 Realizado!

 Teste usando 30 & 800 Realizado!

 Teste usando 30 

In [28]:
# caso gostemos do resultado e desejemos salvar o modelo
Save_model(model,join(MODEL,'rf'))

In [ ]:
from src import MLP